# Acerca de esta competencia

Usted es alguien que está comenzando recientemente en PNL o se ha convertido en un maestro, independientemente de dónde se encuentre en la cadena de aprendizaje, puedo apostar que ha trabajado en el análisis de sentimientos y, si no lo hará, simplemente no puede evitarlo. . ¿Puedes?. El <b> análisis de sentimientos </b> es para PNL. <b> '¿Qué feliz cumpleaños para ti' </b> es para los guitarristas, verdad? Empiece aquí <br>
<br>
En caso de que no esté al tanto del análisis de sentimientos, aquí hay un muy buen artículo: https://towardsdatascience.com/sentiment-analysis-concept-analysis-and-applications-6c94d6f58c17
<br> <br>
Recientemente, Kaggle lanzó una nueva competencia para el COVID-19 Scare, llamada Twitter Sentiment Extraction, sé que es una competencia de análisis de sentimientos de Twitter, pero Kaggle nunca te decepciona, no podría haber sido tan sencillo, después de todo, ha durado dos meses. Entonces, lo que pide esta competencia no son las puntuaciones de sentimiento, sino la parte del tweet (palabra o frase) que refleja el sentimiento. Interesante, ¿no? Esta competencia es especial, así que si quieres mejorar tus habilidades de PNL, esta competencia es para ti.

# Agradecimientos
* https://www.kaggle.com/aashita/word-clouds-of-various-shapes -> FUNCIÓN WORDCLOUDS
* https://www.kaggle.com/rohitsingh9990/ner-training-using-spacy-0-628-lb -> Para comprender cómo entrenar NER espacial en entradas personalizadas


# Acerca de este cuaderno

En este kernel, explicaré brevemente la estructura del conjunto de datos y generaré y analizaré metafunciones. Luego, visualizaré el conjunto de datos usando Matplotlib, seaborn y Plotly para obtener la mayor información posible. También abordaré este problema como un problema NER para construir un modelo
<br> <br>
En caso de que recién esté comenzando con la PNL, aquí hay una guía para abordar casi cualquier problema de PNL del Gran Maestro @Abhishek Thakur
https://www.slideshare.net/abhishekkrthakur/approaching-almost-any-nlp-problem


<b> Este kernel es un trabajo en progreso, y seguiré actualizándolo a medida que avanza la competencia y aprendo más y más cosas sobre los datos </b>

** <span style = "color: Red"> Si encuentra útil este kernel, por favor vote hacia arriba, me motiva a escribir más contenido de calidad ** 

# Importación de necesidades 

In [ ]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

** A continuación se muestra una función auxiliar que genera colores aleatorios que se pueden usar para dar diferentes colores a sus gráficos. Siéntase libre de usarla ** 

In [ ]:
def random_colours(number_of_colors):
    '''
    Simple function for random colours generation.
    Input:
        number_of_colors - integer value indicating the number of colours which are going to be generated.
    Output:
        Color in the following format: ['#E86DA4'] .
    '''
    colors = []
    for i in range(number_of_colors):
        colors.append("#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]))
    return colors

# Leer los datos 

In [ ]:
train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')
ss = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

In [ ]:
print(train.shape)
print(test.shape)

Así que tenemos 27486 tweets en el conjunto de trenes y 3535 tweets en el conjunto de prueba 

In [ ]:
train.info()

Tenemos un valor nulo en el tren, ya que el campo de prueba para el valor es NAN, simplemente lo eliminaremos 

In [ ]:
train.dropna(inplace=True)

In [ ]:
test.info()

No hay valores nulos en el conjunto de prueba 

# Si 

In [ ]:
train.head()

Selected_text es un subconjunto de texto 

In [ ]:
train.describe()

Veamos la distribución de tweets en el tren. 

In [ ]:
temp = train.groupby('sentiment').count()['text'].reset_index().sort_values(by='text',ascending=False)
temp.style.background_gradient(cmap='Purples')

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x='sentiment',data=train)

Dibujemos un gráfico de embudo para una mejor visualización 

In [ ]:
fig = go.Figure(go.Funnelarea(
    text =temp.sentiment,
    values = temp.text,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()

## Qué sabemos actualmente sobre nuestros datos:

Antes de comenzar, veamos algunas cosas que ya sabemos sobre los datos y que nos ayudarán a obtener más conocimientos nuevos:
* Sabemos que selected_text es un subconjunto de texto
* Sabemos que selected_text contiene solo un segmento de texto, es decir, no salta entre dos oraciones. Por ejemplo: - Si el texto es 'Pasé toda la mañana en una reunión con un proveedor, y mi jefe no estaba contento con ellos. Mucha diversión. Tenía otros planes para mi mañana 'El texto seleccionado puede ser' mi jefe no estaba contento con ellos. Mucha diversión 'o' Mucha diversión 'pero no puede ser' Buenos días, vendedor y mi jefe,
* Gracias a esta discusión: https: //www.kaggle.com/c/tweet-sentiment-extraction/discussion/138520 Sabemos que los tweets neutrales tienen una similitud jaccard del 97 por ciento entre el texto y el texto seleccionado
* También como se discutió aquí https://www.kaggle.com/c/tweet-sentiment-extraction/discussion/138272, hay filas donde selected_text comienza entre las palabras y, por lo tanto, selected_texts no siempre tienen sentido y ya que no lo sabemos si la salida del conjunto de prueba contiene estas discrepancias o no, no estamos seguros de que el preprocesamiento y la eliminación de las puntuaciones sea una buena idea o no 

## Generación de metacaracterísticas 

** En las versiones anteriores de este cuaderno, utilicé Número de palabras en el texto seleccionado y el texto principal, Longitud de las palabras en el texto y seleccionadas como meta características principales, pero en el contexto de esta competencia donde tenemos que predecir selected_text que es un subconjunto de texto, las características más útiles para generar serían **: -  * Diferencia en el número de palabras de Selected_text y Text  * Puntuaciones de similitud de Jaccard entre el texto y Selected_text  Por lo tanto, no será útil para nosotros generar características que usamos antes, ya que aquí no tienen importancia.  Para quién no sabe qué es Jaccard Similarity: https://www.geeksforgeeks.org/find-the-jaccard-index-and-jaccard-distance-between-the-two-given-sets/ 

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
results_jaccard=[]

for ind,row in train.iterrows():
    sentence1 = row.text
    sentence2 = row.selected_text

    jaccard_score = jaccard(sentence1,sentence2)
    results_jaccard.append([sentence1,sentence2,jaccard_score])

In [ ]:
jaccard = pd.DataFrame(results_jaccard,columns=["text","selected_text","jaccard_score"])
train = train.merge(jaccard,how='outer')

In [ ]:
train['Num_words_ST'] = train['selected_text'].apply(lambda x:len(str(x).split())) #Number Of words in Selected Text
train['Num_word_text'] = train['text'].apply(lambda x:len(str(x).split())) #Number Of words in main text
train['difference_in_words'] = train['Num_word_text'] - train['Num_words_ST'] #Difference in Number of words text and Selected Text

In [ ]:
train.head()

Veamos la distribución de las metacaracterísticas 

In [ ]:
hist_data = [train['Num_words_ST'],train['Num_word_text']]

group_labels = ['Selected_Text', 'Text']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels,show_curve=False)
fig.update_layout(title_text='Distribution of Number Of words')
fig.update_layout(
    autosize=False,
    width=900,
    height=700,
    paper_bgcolor="LightSteelBlue",
)
fig.show()

* El diagrama de número de palabras es realmente interesante, los tweets que tienen un número de palabras superior a 25 son muy inferiores y, por lo tanto, el diagrama de distribución del número de palabras está sesgado a la derecha 

In [ ]:
plt.figure(figsize=(12,6))
p1=sns.kdeplot(train['Num_words_ST'], shade=True, color="r").set_title('Kernel Distribution of Number Of words')
p1=sns.kdeplot(train['Num_word_text'], shade=True, color="b")

** Ahora será más interesante ver la diferencia en el número de palabras y puntuaciones de jaccard en diferentes sentimientos ** 

In [ ]:
plt.figure(figsize=(12,6))
p1=sns.kdeplot(train[train['sentiment']=='positive']['difference_in_words'], shade=True, color="b").set_title('Kernel Distribution of Difference in Number Of words')
p2=sns.kdeplot(train[train['sentiment']=='negative']['difference_in_words'], shade=True, color="r")

In [ ]:
plt.figure(figsize=(12,6))
sns.distplot(train[train['sentiment']=='neutral']['difference_in_words'],kde=False)

No pude trazar el gráfico kde para tweets neutrales porque la mayoría de los valores para la diferencia en el número de palabras eran cero. Podemos verlo claramente ahora, si hubiéramos usado la función al principio, habríamos sabido que el texto y el texto seleccionado son en su mayoría lo mismo para los tweets neutrales, por lo que siempre es importante tener en cuenta el objetivo final al realizar EDA 

In [ ]:
plt.figure(figsize=(12,6))
p1=sns.kdeplot(train[train['sentiment']=='positive']['jaccard_score'], shade=True, color="b").set_title('KDE of Jaccard Scores across different Sentiments')
p2=sns.kdeplot(train[train['sentiment']=='negative']['jaccard_score'], shade=True, color="r")
plt.legend(labels=['positive','negative'])

No pude trazar kde de jaccard_scores de tweets neutrales por la misma razón, por lo que trazaré una trama de distribución 

In [ ]:
plt.figure(figsize=(12,6))
sns.distplot(train[train['sentiment']=='neutral']['jaccard_score'],kde=False)

Podemos ver algunas tendencias interesantes aquí:  * Los tweets positivos y negativos tienen una alta curtosis y, por lo tanto, los valores se concentran en dos regiones estrechas y de alta densidad.  * Los tweets neutrales tienen un valor de curtosis bajo y su densidad aumenta cerca de los valores de 1  Para aquellos que no saben:  * La curtosis es la medida de cuán pico es una distribución y cuánta propagación está alrededor de ese pico.  * La asimetría mide cuánto se desvía una curva de una distribución normal 

## Conclusión de EDA  * Podemos ver en la gráfica de puntuación de jaccard que hay un pico para la gráfica negativa y positiva alrededor de la puntuación de 1. Eso significa que hay un grupo de tweets donde hay una alta similitud entre el texto y los textos seleccionados, si podemos encontrar esos grupos, entonces podemos predecir texto para textos seleccionados para esos tweets independientemente del segmento  Veamos si podemos encontrar esos grupos, una idea interesante sería revisar los tweets que tienen un número de palabras menor a 3 en el texto, porque allí el texto podría usarse completamente como texto. 

In [ ]:
k = train[train['Num_word_text']<=2]

In [ ]:
k.groupby('sentiment').mean()['jaccard_score']

Podemos ver que existe similitud entre el texto y el texto seleccionado. Veamos más de cerca 

In [ ]:
k[k['sentiment']=='positive']

Por lo tanto, está claro que la mayoría de las veces, el texto se usa como texto seleccionado. Podemos mejorar esto procesando previamente el texto que tiene una longitud de palabra menor a 3. Recordaremos esta información y la usaremos en la construcción de modelos. 

### Limpieza del Corpus  Ahora, antes de sumergirnos en la extracción de información de palabras en texto y texto seleccionado, primero limpiemos los datos 

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
train['text'] = train['text'].apply(lambda x:clean_text(x))
train['selected_text'] = train['selected_text'].apply(lambda x:clean_text(x))

In [ ]:
train.head()

## Palabras más comunes en nuestro texto seleccionado de destino 

In [ ]:
train['temp_list'] = train['selected_text'].apply(lambda x:str(x).split())
top = Counter([item for sublist in train['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

In [ ]:
fig = px.bar(temp, x="count", y="Common_words", title='Commmon Words in Selected Text', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

¡Vaya! Mientras limpiamos nuestro conjunto de datos, no eliminamos las palabras vacías y, por lo tanto, podemos ver que la palabra más común es 'para'. Intentemos de nuevo después de eliminar las palabras vacías 

In [ ]:
def remove_stopword(x):
    return [y for y in x if y not in stopwords.words('english')]
train['temp_list'] = train['temp_list'].apply(lambda x:remove_stopword(x))

In [ ]:
top = Counter([item for sublist in train['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp = temp.iloc[1:,:]
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Purples')

In [ ]:
fig = px.treemap(temp, path=['Common_words'], values='count',title='Tree of Most Common Words')
fig.show()

# Palabras más comunes en el texto  Veamos también las palabras más comunes en Text 

In [ ]:
train['temp_list1'] = train['text'].apply(lambda x:str(x).split()) #List of words in every row for text
train['temp_list1'] = train['temp_list1'].apply(lambda x:remove_stopword(x)) #Removing Stopwords

In [ ]:
top = Counter([item for sublist in train['temp_list1'] for item in sublist])
temp = pd.DataFrame(top.most_common(25))
temp = temp.iloc[1:,:]
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

Entonces, las dos primeras palabras comunes fueron I'm, así que la eliminé y tomé datos de la segunda fila 

In [ ]:
fig = px.bar(temp, x="count", y="Common_words", title='Commmon Words in Text', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

Entonces podemos ver que las palabras más comunes en el texto seleccionado y el texto son casi las mismas, lo cual era obvio 

# Palabras más comunes Sentimientos Sabio  Veamos las palabras más comunes en diferentes sentimientos. 

In [ ]:
Positive_sent = train[train['sentiment']=='positive']
Negative_sent = train[train['sentiment']=='negative']
Neutral_sent = train[train['sentiment']=='neutral']

In [ ]:
#MosT common positive words
top = Counter([item for sublist in Positive_sent['temp_list'] for item in sublist])
temp_positive = pd.DataFrame(top.most_common(20))
temp_positive.columns = ['Common_words','count']
temp_positive.style.background_gradient(cmap='Greens')

In [ ]:
fig = px.bar(temp_positive, x="count", y="Common_words", title='Most Commmon Positive Words', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

In [ ]:
#MosT common negative words
top = Counter([item for sublist in Negative_sent['temp_list'] for item in sublist])
temp_negative = pd.DataFrame(top.most_common(20))
temp_negative = temp_negative.iloc[1:,:]
temp_negative.columns = ['Common_words','count']
temp_negative.style.background_gradient(cmap='Reds')

In [ ]:
fig = px.treemap(temp_negative, path=['Common_words'], values='count',title='Tree Of Most Common Negative Words')
fig.show()

In [ ]:
#MosT common Neutral words
top = Counter([item for sublist in Neutral_sent['temp_list'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Common_words','count']
temp_neutral.style.background_gradient(cmap='Reds')

In [ ]:
fig = px.bar(temp_neutral, x="count", y="Common_words", title='Most Commmon Neutral Words', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

In [ ]:
fig = px.treemap(temp_neutral, path=['Common_words'], values='count',title='Tree Of Most Common Neutral Words')
fig.show()

* Podemos ver palabras como get, go, dont, got, u, can, lol, like son comunes en los tres segmentos. Eso es interesante porque palabras como no y no puedo son más de naturaleza negativa y palabras como lol son más de naturaleza positiva. ¿Significa esto que nuestros datos están etiquetados incorrectamente, tendremos más información sobre esto después del análisis de N-gram  * Será interesante ver la palabra única para diferentes sentimientos. 

## Veamos palabras únicas en cada segmento  Veremos palabras únicas en cada segmento en el siguiente orden:  * Positivo  * Negativo  * Neutral 

In [ ]:
raw_text = [word for word_list in train['temp_list1'] for word in word_list]

In [ ]:
def words_unique(sentiment,numwords,raw_words):
    '''
    Input:
        segment - Segment category (ex. 'Neutral');
        numwords - how many specific words do you want to see in the final result; 
        raw_words - list  for item in train_data[train_data.segments == segments]['temp_list1']:
    Output: 
        dataframe giving information about the name of the specific ingredient and how many times it occurs in the chosen cuisine (in descending order based on their counts)..

    '''
    allother = []
    for item in train[train.sentiment != sentiment]['temp_list1']:
        for word in item:
            allother .append(word)
    allother  = list(set(allother ))
    
    specificnonly = [x for x in raw_text if x not in allother]
    
    mycounter = Counter()
    
    for item in train[train.sentiment == sentiment]['temp_list1']:
        for word in item:
            mycounter[word] += 1
    keep = list(specificnonly)
    
    for word in list(mycounter):
        if word not in keep:
            del mycounter[word]
    
    Unique_words = pd.DataFrame(mycounter.most_common(numwords), columns = ['words','count'])
    
    return Unique_words

### Tweets positivos 

In [ ]:
Unique_Positive= words_unique('positive', 20, raw_text)
print("The top 20 unique words in Positive Tweets are:")
Unique_Positive.style.background_gradient(cmap='Greens')

In [ ]:
fig = px.treemap(Unique_Positive, path=['words'], values='count',title='Tree Of Unique Positive Words')
fig.show()

In [ ]:
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize=(16,10))
my_circle=plt.Circle((0,0), 0.7, color='white')
plt.pie(Unique_Positive['count'], labels=Unique_Positive.words, colors=Pastel1_7.hex_colors)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('DoNut Plot Of Unique Positive Words')
plt.show()

In [ ]:
Unique_Negative= words_unique('negative', 10, raw_text)
print("The top 10 unique words in Negative Tweets are:")
Unique_Negative.style.background_gradient(cmap='Reds')

In [ ]:
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize=(16,10))
my_circle=plt.Circle((0,0), 0.7, color='white')
plt.rcParams['text.color'] = 'black'
plt.pie(Unique_Negative['count'], labels=Unique_Negative.words, colors=Pastel1_7.hex_colors)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('DoNut Plot Of Unique Negative Words')
plt.show()

In [ ]:
Unique_Neutral= words_unique('neutral', 10, raw_text)
print("The top 10 unique words in Neutral Tweets are:")
Unique_Neutral.style.background_gradient(cmap='Oranges')

In [ ]:
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize=(16,10))
my_circle=plt.Circle((0,0), 0.7, color='white')
plt.pie(Unique_Neutral['count'], labels=Unique_Neutral.words, colors=Pastel1_7.hex_colors)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('DoNut Plot Of Unique Neutral Words')
plt.show()

** Al observar las palabras únicas de cada sentimiento, ahora tenemos mucha más claridad sobre los datos, estas palabras únicas son determinantes muy fuertes del sentimiento de los tweets ** 

## Es hora de nubes de palabras  Construiremos nubes de palabras en el siguiente orden:  * WordCloud de tweets neutrales  * WordCloud de tweets positivos  * WordCloud de tweets negativos 

In [ ]:
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=100, figure_size=(24.0,16.0), color = 'white',
                   title = None, title_size=40, image_color=False):
    stopwords = set(STOPWORDS)
    more_stopwords = {'u', "im"}
    stopwords = stopwords.union(more_stopwords)

    wordcloud = WordCloud(background_color=color,
                    stopwords = stopwords,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    width=400, 
                    height=200,
                    mask = mask)
    wordcloud.generate(str(text))
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout()  
d = '/kaggle/input/masks-for-wordclouds/'

He agregado más palabras como im, u (que decimos que estaban allí en las palabras más comunes, perturbando nuestro análisis) como palabras vacías 

#### NUBE DE PALABRAS DE TWEETS NEUTRALES  Ya hemos visualizado nuestras palabras negativas más comunes, pero las nubes de palabras nos brindan mucha más claridad 

In [ ]:
pos_mask = np.array(Image.open(d+ 'twitter_mask.png'))
plot_wordcloud(Neutral_sent.text,mask=pos_mask,color='white',max_font_size=100,title_size=30,title="WordCloud of Neutral Tweets")

In [ ]:
plot_wordcloud(Positive_sent.text,mask=pos_mask,title="Word Cloud Of Positive tweets",title_size=30)

In [ ]:
plot_wordcloud(Negative_sent.text,mask=pos_mask,title="Word Cloud of Negative Tweets",color='white',title_size=30)

# Modelado

Esta es la primera competencia de kaggle, en la que estoy participando y este podría ser el caso de muchos de nosotros. Debido a la estructura única del planteamiento del problema, es difícil para cualquier principiante o novato de competencias responder a la pregunta "¿Qué modelo to Use "?. Mis pensamientos iniciales fueron que esta competencia no es para mí y ya terminé aquí, pero luego recordé algo, estuve en el KaggleDays Meetup Delhi este año y tuve esta maravillosa oportunidad de conocer al Gran Maestro Abhishek Thakur y durante el Sesión de preguntas y respuestas Le pregunté que las competiciones de kaggle son tan diversas, únicas, requieren muchos conocimientos previos y, por lo tanto, da miedo participar, a lo que respondió y cito "¡Aterrador, sí! aprende si no vas a participar ".

Así que aquí estoy luchando para abrirme camino a través de esta competencia y tratando de aprender cosas diferentes e insto a todos a hacer lo mismo, puede que no esté tan bien establecido para dar consejos, pero realmente quería compartir esa historia para motivar a la gente.

Después de pasar por los foros de discusión, seguir los consejos de los expertos y ver el tutorial de Abhishek Sir anoche, este problema se puede modelar de la siguiente manera:
* Reconocimiento de entidad nombrada
* Problema de preguntas y respuestas
* También encontré un enfoque simple compartido por Nick en su hermoso kernel donde tiene el concepto de Gini Impurity para dar peso a las palabras presentes en los tweets y luego predecir usando el peso de esas palabras: https://www.kaggle.com/ nkoprowicz / a-simple-solution-using-only-word-count / notebook. Compruébelo.
* Otras ideas de modelado: - https://www.kaggle.com/c/tweet-sentiment-extraction/discussion/139803 -> Aquí hay una muy buena idea
* Otra idea útil: - https://www.kaggle.com/c/tweet-sentiment-extraction/discussion/139335

Recursos:
* Para problemas de modelado como NER: https://www.kaggle.com/rohitsingh9990/ner-training-using-spacy-0-628-lb
* Para problemas de modelado AS Q&A: https://www.kaggle.com/jonathanbesomi/question-answering-starter-pack ---> Esta es una guía completa y desde cero 

## 1) Modelando el problema como NER

El reconocimiento de entidades con nombre (NER) es un problema estándar de PNL que implica detectar entidades con nombre (personas, lugares, organizaciones, etc.) de un fragmento de texto y clasificarlas en un conjunto predefinido de categorías.
Para comprender NER, aquí hay un muy buen artículo: https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da

Usaremos spacy para crear nuestro propio modelo o modelos NER personalizados (separados para cada Sentiment). La motivación para este enfoque es, por supuesto, el kernel compartido por Rohit Singh, así que si encuentra útil su kernel, por favor vote por él.

¿Qué será diferente con mi solución?
* Usaré text como selected_text para todos los tweets neutrales debido a su gran similitud con jaccard
* También usaré text como selected_text para todos los tweets que tengan un número de palabras menor a 3 en el texto como se explicó antes
* Entrenaré dos modelos diferentes para tweets positivos y negativos
* No preprocesaré los datos porque el texto seleccionado contiene texto sin formato 

In [ ]:
df_train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
df_test = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')
df_submission = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

In [ ]:
df_train['Num_words_text'] = df_train['text'].apply(lambda x:len(str(x).split())) #Number Of words in main Text in train set

In [ ]:
df_train = df_train[df_train['Num_words_text']>=3]

** Para una comprensión completa de cómo entrenar Spacy NER con entradas personalizadas, lea la documentación de Spacy junto con la presentación del código en este cuaderno: https://spacy.io/usage/training#ner Siga las instrucciones de Actualización de Spacy NER * * 

In [ ]:
def save_model(output_dir, nlp, new_model_name):
    ''' This Function Saves model to 
    given output directory'''
    
    output_dir = f'../working/{output_dir}'
    if output_dir is not None:        
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        nlp.meta["name"] = new_model_name
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

In [ ]:
# pass model = nlp if you want to train on top of existing model 

def train(train_data, output_dir, n_iter=20, model=None):
    """Load the model, set up the pipeline and train the entity recognizer."""
    ""
    if model is not None:
        nlp = spacy.load(output_dir)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
    
    # add labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        if model is None:
            nlp.begin_training()
        else:
            nlp.resume_training()


        for itn in tqdm(range(n_iter)):
            random.shuffle(train_data)
            batches = minibatch(train_data, size=compounding(4.0, 500.0, 1.001))    
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts,  # batch of texts
                            annotations,  # batch of annotations
                            drop=0.5,   # dropout - make it harder to memorise data
                            losses=losses, 
                            )
            print("Losses", losses)
    save_model(output_dir, nlp, 'st_ner')

In [ ]:
def get_model_out_path(sentiment):
    '''
    Returns Model output path
    '''
    model_out_path = None
    if sentiment == 'positive':
        model_out_path = 'models/model_pos'
    elif sentiment == 'negative':
        model_out_path = 'models/model_neg'
    return model_out_path

In [ ]:
def get_training_data(sentiment):
    '''
    Returns Trainong data in the format needed to train spacy NER
    '''
    train_data = []
    for index, row in df_train.iterrows():
        if row.sentiment == sentiment:
            selected_text = row.selected_text
            text = row.text
            start = text.find(selected_text)
            end = start + len(selected_text)
            train_data.append((text, {"entities": [[start, end, 'selected_text']]}))
    return train_data

#### Modelos de entrenamiento para tweets positivos y negativos 

In [ ]:
sentiment = 'positive'

train_data = get_training_data(sentiment)
model_path = get_model_out_path(sentiment)
# For DEmo Purposes I have taken 3 iterations you can train the model as you want
train(train_data, model_path, n_iter=3, model=None)

In [ ]:
sentiment = 'negative'

train_data = get_training_data(sentiment)
model_path = get_model_out_path(sentiment)

train(train_data, model_path, n_iter=3, model=None)

### Predicción con el modelo entrenado 

In [ ]:
def predict_entities(text, model):
    doc = model(text)
    ent_array = []
    for ent in doc.ents:
        start = text.find(ent.text)
        end = start + len(ent.text)
        new_int = [start, end, ent.label_]
        if new_int not in ent_array:
            ent_array.append([start, end, ent.label_])
    selected_text = text[ent_array[0][0]: ent_array[0][1]] if len(ent_array) > 0 else text
    return selected_text

In [ ]:
selected_texts = []
MODELS_BASE_PATH = '../input/tse-spacy-model/models/'

if MODELS_BASE_PATH is not None:
    print("Loading Models  from ", MODELS_BASE_PATH)
    model_pos = spacy.load(MODELS_BASE_PATH + 'model_pos')
    model_neg = spacy.load(MODELS_BASE_PATH + 'model_neg')
        
    for index, row in df_test.iterrows():
        text = row.text
        output_str = ""
        if row.sentiment == 'neutral' or len(text.split()) <= 2:
            selected_texts.append(text)
        elif row.sentiment == 'positive':
            selected_texts.append(predict_entities(text, model_pos))
        else:
            selected_texts.append(predict_entities(text, model_neg))
        
df_test['selected_text'] = selected_texts

In [ ]:
df_submission['selected_text'] = df_test['selected_text']
df_submission.to_csv("submission.csv", index=False)
display(df_submission.head(10))

# Notas finales
Kaggle siempre proporciona muchos días para una competencia que uno puede utilizar para aprender y crecer.Como prometí, presenté mi primer modelo, junto con una explicación, puede leer la documentación de Spacy y el kernel de Rohit Singh ya que todo el código proviene de su. si comprende cualquier parte del código, no dude en comentar y preguntar, intentaré resolverlo.
Como esta es mi primera competencia, también estoy aprendiendo a lo largo del camino, volveré con ideas más originales y algunos modelos geniales a medida que aprenda más y más sobre preguntas / respuestas, otras técnicas diferentes, varias formas de BERT y los datos en sí.

** Gracias por el enorme amor y aprecio, lamento no haber actualizado el kernel con el enfoque de preguntas y respuestas, todavía estoy aprendiendo todas las técnicas necesarias, ¡lo actualizaré pronto! **
<br> <br> ¡MANTÉNGASE SINTONIZADO!

<span style = "color: Red"> Espero que les haya gustado mi kernel. Un upvote es un gesto de agradecimiento y aliento que me llena de energía para seguir mejorando mis esfuerzos, sea amable de mostrar uno ;-) 